In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.pipeline import Pipeline
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.linear_model import SGDClassifier, LogisticRegression

import time
 
now = time.localtime()
print("%04d/%02d/%02d %02d:%02d" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min))

end = time.localtime()
print("%04d/%02d/%02d %02d:%02d" % (end.tm_year, end.tm_mon, end.tm_mday, end.tm_hour, end.tm_min))


2020/11/26 21:14
2020/11/26 21:14


In [2]:
start = time.localtime()

end = time.localtime()
print("%04d/%02d/%02d %02d:%02d" % (start.tm_year, start.tm_mon, start.tm_mday, start.tm_hour, start.tm_min))
print("%04d/%02d/%02d %02d:%02d" % (end.tm_year, end.tm_mon, end.tm_mday, end.tm_hour, end.tm_min))


2020/11/26 21:14
2020/11/26 21:14


In [3]:
train = pd.read_csv('open/train.csv')
test = pd.read_csv('open/test_x.csv')
submiss=pd.read_csv("open/sample_submission.csv",index_col = [0])

In [4]:
X_Train=train['text'].str.replace('[^a-zA-Z0-9]', ' ')
y_train=train['author']
X_Test=test['text'].str.replace('[^a-zA-Z0-9]', ' ')

In [5]:
x_train, x_test, y_train, y_test = train_test_split(X_Train, y_train, test_size=0.2)

## TfidfVectorizer

In [53]:
## Multinomial Naive Bayes Classifier ##
# Build pipeline
classifier = Pipeline([('vect', TfidfVectorizer(lowercase=False)),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
])

# parameter tuning with grid search
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 2)],
              'vect__max_df': [0.2, 0.25],
              'vect__min_df': [1],
              'vect__analyzer' : ['word'],
              'clf__alpha': [0.031],
}
gs_clf = GridSearchCV(classifier, parameters, n_jobs=-1, verbose=1, cv=2)
gs_clf.fit(x_train, y_train)

best_parameters = gs_clf.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

# Predicting the Test set results
y_pred_proba = gs_clf.predict_proba(x_test)
print(log_loss(y_test, y_pred_proba))
# y_pred_proba = gs_clf.predict_proba(X_Test)

Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    8.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    8.2s finished


	clf__alpha: 0.031
	vect__analyzer: 'word'
	vect__max_df: 0.25
	vect__min_df: 1
	vect__ngram_range: (1, 2)
0.61324808532512


In [55]:
## Multinomial Naive Bayes Classifier ##
# Build pipeline
classifier = Pipeline([('vect', TfidfVectorizer(lowercase=False)),
                      ('tfidf', TfidfTransformer()),
                      ('clf', CalibratedClassifierCV(MultinomialNB(alpha=0.05), method='isotonic')),
])

# parameter tuning with grid search
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 2)],
              'vect__max_df': (0.4, 0.5),
#               'vect__min_df': [1],
              'vect__analyzer' : ['word'],
#               'clf__alpha': (0.016, 0.019),
}
gs_clf = GridSearchCV(classifier, parameters, n_jobs=-1, verbose=1, cv=2)
gs_clf.fit(x_train, y_train)

best_parameters = gs_clf.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

# Predicting the Test set results
y_pred_proba = gs_clf.predict_proba(x_test)
print(log_loss(y_test, y_pred_proba))
# y_pred_proba = gs_clf.predict_proba(X_Test)

Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    8.8s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    8.8s finished


	vect__analyzer: 'word'
	vect__max_df: 0.5
	vect__ngram_range: (1, 2)
0.6087135875789627


In [61]:
## Multinomial Naive Bayes Classifier ##
# Build pipeline
classifier = Pipeline([('vect', TfidfVectorizer(lowercase=False)),
                      ('tfidf', TfidfTransformer()),
                      ('clf', CalibratedClassifierCV(BernoulliNB(alpha=0.02), method='isotonic')),
])

# parameter tuning with grid search
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 2)],
              'vect__max_df': [0.03],
              'vect__min_df': [1],
              'vect__analyzer' : ['word'],
#               'clf__alpha': (0.016, 0.019),
}
gs_clf = GridSearchCV(classifier, parameters, n_jobs=-1, verbose=1, cv=2)
gs_clf.fit(x_train, y_train)

best_parameters = gs_clf.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

# Predicting the Test set results
y_pred_proba = gs_clf.predict_proba(x_test)
print(log_loss(y_test, y_pred_proba))
# y_pred_proba = gs_clf.predict_proba(X_Test)

Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    7.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    7.5s finished


	vect__analyzer: 'word'
	vect__max_df: 0.03
	vect__min_df: 1
	vect__ngram_range: (1, 2)
0.643483366095326


In [62]:
## Multinomial Naive Bayes Classifier ##
# Build pipeline
classifier = Pipeline([('vect', TfidfVectorizer(lowercase=False)),
                      ('tfidf', TfidfTransformer()),
                      ('clf', CalibratedClassifierCV(SGDClassifier(loss='modified_huber', alpha=1e-4, max_iter=10000, tol=1e-4), method='sigmoid')),
])

# parameter tuning with grid search
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 2), (1, 3)],
#               'vect__max_df': (0.5, 0.6),
#               'vect__min_df': [1],
              'vect__analyzer' : ['word'],
#               'clf__alpha': (0.016, 0.019),
}
gs_clf = GridSearchCV(classifier, parameters, n_jobs=-1, verbose=1, cv=2)
gs_clf.fit(x_train, y_train)

best_parameters = gs_clf.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

# Predicting the Test set results
y_pred_proba = gs_clf.predict_proba(x_test)
print(log_loss(y_test, y_pred_proba))
# y_pred_proba = gs_clf.predict_proba(X_Test)

Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   39.6s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   39.6s finished


	vect__analyzer: 'word'
	vect__ngram_range: (1, 2)
0.6120074265496387


In [52]:
## Multinomial Naive Bayes Classifier ##
# Build pipeline
classifier = Pipeline([('vect', TfidfVectorizer(lowercase=False)),
                      ('tfidf', TfidfTransformer()),
                      ('clf', LogisticRegression(C=50, max_iter=200)),
])

# parameter tuning with grid search
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 2), (1, 3)],
#               'vect__max_df': (0.5, 0.6),
#               'vect__min_df': [1],
              'vect__analyzer' : ['word'],
#               'clf__alpha': (0.016, 0.019),
}
gs_clf = GridSearchCV(classifier, parameters, n_jobs=-1, verbose=1, cv=2)
gs_clf.fit(x_train, y_train)

best_parameters = gs_clf.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

# Predicting the Test set results
y_pred_proba = gs_clf.predict_proba(x_test)
print(log_loss(y_test, y_pred_proba))
# y_pred_proba = gs_clf.predict_proba(X_Test)

Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:  4.6min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:  4.6min finished
C:\Users\id272\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


	vect__analyzer: 'word'
	vect__ngram_range: (1, 2)
0.5759024441009035


In [ ]:
# max_iter=200 -> 0.5817073749174595   


# C=10 -> 0.6128354782666344
# C=20 -> 0.5903689734636244
# C=30 -> 0.5817073749174595
# C=40 -> 0.5781535999295956
# C=45 -> 0.5793662895008217

# C=50 -> 0.5759024441009035
# C=60 -> 0.5789007294252998



## CountVectorizer

In [51]:
## Multinomial Naive Bayes Classifier ##
# Build pipeline
classifier = Pipeline([('vect', CountVectorizer(lowercase=False)),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB(alpha=0.03)),
])

# parameter tuning with grid search
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 2), (1, 3)],
#               'vect__max_df': (0.5, 0.6),
#               'vect__min_df': [1],
              'vect__analyzer' : ['word'],
#               'clf__alpha': (0.016, 0.019),
}
gs_clf = GridSearchCV(classifier, parameters, n_jobs=-1, verbose=1, cv=2)
gs_clf.fit(x_train, y_train)

best_parameters = gs_clf.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

# Predicting the Test set results
y_pred_proba = gs_clf.predict_proba(x_test)
print(log_loss(y_test, y_pred_proba))
# y_pred_proba = gs_clf.predict_proba(X_Test)

Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   29.8s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   29.8s finished


	vect__analyzer: 'word'
	vect__ngram_range: (1, 2)
0.5908551753092789


In [52]:
## Multinomial Naive Bayes Classifier ##
# Build pipeline
classifier = Pipeline([('vect', CountVectorizer(lowercase=False)),
                      ('tfidf', TfidfTransformer()),
                      ('clf', CalibratedClassifierCV(MultinomialNB(alpha=0.03), method='isotonic')),
])

# parameter tuning with grid search
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 2), (1, 3)],
#               'vect__max_df': (0.5, 0.6),
#               'vect__min_df': [1],
              'vect__analyzer' : ['word'],
#               'clf__alpha': (0.016, 0.019),
}
gs_clf = GridSearchCV(classifier, parameters, n_jobs=-1, verbose=1, cv=2)
gs_clf.fit(x_train, y_train)

best_parameters = gs_clf.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

# Predicting the Test set results
y_pred_proba = gs_clf.predict_proba(x_test)
print(log_loss(y_test, y_pred_proba))
# y_pred_proba = gs_clf.predict_proba(X_Test)

Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   31.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   31.1s finished


	vect__analyzer: 'word'
	vect__ngram_range: (1, 2)
0.572322278123051


In [53]:
## Multinomial Naive Bayes Classifier ##
# Build pipeline
classifier = Pipeline([('vect', CountVectorizer(lowercase=False)),
                      ('tfidf', TfidfTransformer()),
                      ('clf', CalibratedClassifierCV(BernoulliNB(alpha=0.03), method='isotonic')),
])

# parameter tuning with grid search
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 2), (1, 3)],
#               'vect__max_df': (0.5, 0.6),
#               'vect__min_df': [1],
              'vect__analyzer' : ['word'],
#               'clf__alpha': (0.016, 0.019),
}
gs_clf = GridSearchCV(classifier, parameters, n_jobs=-1, verbose=1, cv=2)
gs_clf.fit(x_train, y_train)

best_parameters = gs_clf.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

# Predicting the Test set results
y_pred_proba = gs_clf.predict_proba(x_test)
print(log_loss(y_test, y_pred_proba))
# y_pred_proba = gs_clf.predict_proba(X_Test)

Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   35.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   35.2s finished


	vect__analyzer: 'word'
	vect__ngram_range: (1, 2)
0.6481480500928002


In [54]:
## Multinomial Naive Bayes Classifier ##
# Build pipeline
classifier = Pipeline([('vect', CountVectorizer(lowercase=False)),
                      ('tfidf', TfidfTransformer()),
                      ('clf', CalibratedClassifierCV(SGDClassifier(loss='modified_huber', alpha=1e-4, max_iter=10000, tol=1e-4), method='sigmoid')),
])

# parameter tuning with grid search
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 2), (1, 3)],
#               'vect__max_df': (0.5, 0.6),
#               'vect__min_df': [1],
              'vect__analyzer' : ['word'],
#               'clf__alpha': (0.016, 0.019),
}
gs_clf = GridSearchCV(classifier, parameters, n_jobs=-1, verbose=1, cv=2)
gs_clf.fit(x_train, y_train)

best_parameters = gs_clf.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

# Predicting the Test set results
y_pred_proba = gs_clf.predict_proba(x_test)
print(log_loss(y_test, y_pred_proba))
# y_pred_proba = gs_clf.predict_proba(X_Test)

Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   39.9s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:   39.9s finished


	vect__analyzer: 'word'
	vect__ngram_range: (1, 2)
0.6447651217453803


In [6]:
## Multinomial Naive Bayes Classifier ##
# Build pipeline
classifier = Pipeline([('vect', CountVectorizer(lowercase=False)),
                      ('tfidf', TfidfTransformer()),
                      ('clf', LogisticRegression(C=30, max_iter = 200)),
])

# parameter tuning with grid search
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 2), (1, 3)],
#               'vect__max_df': (0.5, 0.6),
#               'vect__min_df': [1],
              'vect__analyzer' : ['word'],
#               'clf__alpha': (0.016, 0.019),
}
gs_clf = GridSearchCV(classifier, parameters, n_jobs=-1, verbose=1, cv=2)
gs_clf.fit(x_train, y_train)

best_parameters = gs_clf.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

# Predicting the Test set results
y_pred_proba = gs_clf.predict_proba(x_test)
print(log_loss(y_test, y_pred_proba))
# y_pred_proba = gs_clf.predict_proba(X_Test)

Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:  6.7min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:  6.7min finished
C:\Users\id272\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


	vect__analyzer: 'word'
	vect__ngram_range: (1, 2)
0.5996524690884899


In [ ]:
# 0.6507361092389561
# 0.5996524690884899

In [ ]:
start = time.localtime()
print("%04d/%02d/%02d %02d:%02d" % (start.tm_year, start.tm_mon, start.tm_mday, start.tm_hour, start.tm_min))
# Final Model
# XGBoost
def runXGB(train_X, train_y, test_X, test_y=None, test_X2=None, seed_val=0, child=1, colsample=0.3):
    param = {}
    param['objective'] = 'multi:softprob'
    param['eta'] = 0.1
    param['max_depth'] = 5
#     param['silent'] = 1
    param['num_class'] = 5
    param['eval_metric'] = "mlogloss"
    param['min_child_weight'] = child
    param['subsample'] = 0.8
    param['colsample_bytree'] = colsample
    param['seed'] = seed_val
    num_rounds = 2000

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=50, verbose_eval=20)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)

    pred_test_y = model.predict(xgtest, ntree_limit = model.best_ntree_limit)
    if test_X2 is not None:
        xgtest2 = xgb.DMatrix(test_X2)
        pred_test_y2 = model.predict(xgtest2, ntree_limit = model.best_ntree_limit)
    return pred_test_y, pred_test_y2, model

def do(train, test, Y_train):
    drop_columns=['index', "text"]
    x_train = train.drop(drop_columns+['author'],axis=1)
    x_test = test.drop(drop_columns,axis=1)
    y_train = Y_train
    
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=32143233)
    cv_scores = []
    pred_full_test = 0
    pred_train = np.zeros([x_train.shape[0], 5])
    for dev_index, val_index in kf.split(x_train):
        dev_X, val_X = x_train.loc[dev_index], x_train.loc[val_index]
        dev_y, val_y = y_train[dev_index], y_train[val_index]
        pred_val_y, pred_test_y, model = runXGB(dev_X, dev_y, val_X, val_y, x_test, seed_val=0, colsample=0.7)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index,:] = pred_val_y
        cv_scores.append(metrics.log_loss(val_y, pred_val_y))
    print("cv score : ", cv_scores)
    print("Mean cv score : ", np.mean(cv_scores))
    return pred_full_test/5
result = do(train, test, Y_train)

end = time.localtime()
print("%04d/%02d/%02d %02d:%02d" % (start.tm_year, start.tm_mon, start.tm_mday, start.tm_hour, start.tm_min))
print("%04d/%02d/%02d %02d:%02d" % (end.tm_year, end.tm_mon, end.tm_mday, end.tm_hour, end.tm_min))

In [21]:
submiss['0']=y_pred_proba[:,0]
submiss['1']=y_pred_proba[:,1]
submiss['2']=y_pred_proba[:,2]
submiss['3']=y_pred_proba[:,3]
submiss['4']=y_pred_proba[:,4]
submiss.to_csv('practice.csv')